# Testing

[Link to the website](https://www.thomasnet.com/)

In [40]:
import requests
from bs4 import BeautifulSoup
from traceback import print_exc

from datetime import datetime
import time

import numpy as np 
import pandas as pd
import math

In [21]:
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36"
}

payload = {
    # "WTZO": "Find Suppliers",
    "cov": "NA",
    "heading": 21650809,
    "searchsource": "suppliers",
    "searchterm": "hydraulic cylinders",
    # "searchx": "true",
    "what": "hydraulic cylinders",
    # "which": "prod",
    "pg": 1
}

BASE_URL = "https://www.thomasnet.com/nsearch.html"

In [22]:
page = requests.get(BASE_URL, params=payload)
soup = BeautifulSoup(page.text,"lxml")

total_suppliers = soup.find('p',class_="supplier-search-results__subheader").findAll('b')[-1].text
print(f"{total_suppliers} suppliers found")

n_suppliers = len(soup.findAll('div',class_="supplier-search-results__card"))
print(f"{n_suppliers} found on this page")

number_of_pages = math.ceil(float(total_suppliers)/float(n_suppliers))
print(f"Total Pages: {number_of_pages}")

772 suppliers found
25 found on this page
Total Pages: 31


In [24]:
def generate_payloads(keyword: str, page_num: int):
    payloads = []
    for i in range(page_num):
        payload = {
            "cov": "NA",
            "heading": 21650809,
            "searchsource": "suppliers",
            "searchterm": keyword,
            "what": keyword,
            "pg": i+1
        }
        payloads.append(payload)

    return payloads

x = generate_payloads('hydraulic cylinders',number_of_pages)

In [56]:
collected_data = []

keyword = 'hydraulic cylinders'
page = requests.get(BASE_URL, params=x[7])
soup = BeautifulSoup(page.text,"lxml")

suppliers = soup.findAll('div',class_="supplier-search-results__card")
for sup in suppliers:
    card_data = {
        "company_id":"",
        "company_name":"",
        "company_type":"",
        "annual_revenue":"",
        "year_founded":"",
        "num_employees":"",
        "location":"",
        "company_url":"",
        "brands":"",
        "description":"",
        "url":"",
        "telephone":"",
        "searchterm":keyword
    }
    try:
        header = sup.find('header',class_='profile-card__header')
        card_data['company_id'] = eval(sup.get('data-impression-tracking'))['company_id']
        card_data['company_name'] = header.find('h2',class_='profile-card__title').text.strip()
        card_data['url'] = 'https://www.thomasnet.com'+header.find('h2',class_='profile-card__title').find('a').get('href')
        card_data['telephone'] = sup.find('a',{'data-conversion_action':'Call'}).get('href')

        sup_data = sup.find('div',class_='profile-card__supplier-data')
        card_data['location'] = sup_data.find('span',class_='profile-card__location').text.replace('ico-map','').strip()
        card_data['company_type'] = sup_data.find('span',{'data-content':'Company Type'}).text.strip()
        try:
            card_data['annual_revenue'] = sup_data.find('span',{'data-content':'Annual Revenue'}).text.strip()
        except:
            pass
        try:
            card_data['num_employees'] = sup_data.find('span',{'data-content':'Number of Employees'}).text.strip()
        except:
            pass
        try:
            card_data['year_founded'] = sup_data.find('span',{'data-content':'Year Founded'}).text.strip()
        except:
            pass

        content = sup.find('div',class_='profile-card__content')
        card_data['description'] = content.findAll('p')[0].text.strip()
        try:
            card_data['company_url'] = content.find('p',{'class':'profile-card_web-link-wrap'}).find('a').get('href')
        except:
            pass
        try:
            card_data['brands'] = content.find('p',{'class':'profile-card__brands__body'}).text.strip()
        except:
            pass
    except Exception as e:
        print("Error encountered while extraction of data\n",print_exc())
        pass

    collected_data.append(card_data)

df = pd.DataFrame(collected_data)
df

,company_id,company_name,company_type,annual_revenue,year_founded,num_employees,location,company_url,brands,description,url,telephone,searchterm
0,30829630,Wulfs Custom Welding Inc,Manufacturer*,,,,"Springfield, NE",,,Manufacturer of hand tools and equipment. Prod...,https://www.thomasnet.com/profile/30829630/wul...,tel://402-253-2772,hydraulic cylinders
1,1115028,Helac Corp.,Manufacturer*,$25 - 49.9 Mil,1968,100-199,"Enumclaw, WA",,,Manufacturer & designer of hydraulic cylinders...,https://www.thomasnet.com/profile/01115028/hel...,tel://800-327-2589,hydraulic cylinders
2,1308693,Carlson Hydraulics,"Manufacturer*, Custom Manufacturer",$5 - 9.9 Mil,1985,10-49,"Wichita, KS",,,Manufacturer of welded hydraulic cylinders. Fe...,https://www.thomasnet.com/profile/01308693/car...,tel://888-378-3667,hydraulic cylinders
3,20055772,"Southwest Seal & Supply, Inc.",Distributor*,$250 Mil. and over,1979,10-49,Locations,,,Distributor of industrial gas hydraulic cylind...,https://www.thomasnet.com/branchloc.html?cid=2...,tel://505-889-8828,hydraulic cylinders
4,30703197,"Anker-Holth Ltd., Div. of Magnum Integrated Te...",Manufacturer*,$10 - 24.9 Mil,1953,10-49,"Brampton, ON",,,Manufacturer of high performance hydraulic sys...,https://www.thomasnet.com/profile/30703197/ank...,tel://800-387-3834,hydraulic cylinders
5,360899,General Engineering Co.,"Custom Manufacturer*, Distributor, Manufacture...",$10 - 24.9 Mil,1948,50-99,"Abingdon, VA",,,ISO 9001:2000 certified manufacturer of custom...,https://www.thomasnet.com/profile/00360899/gen...,tel://276-628-6068,hydraulic cylinders
6,95684,Hydraulic Service & Supply Co.,"Distributor*, Service Company",Under $1 Mil,1956,10-49,"Dallas, TX",,,Distributor For Power Team Hydraulic Component...,https://www.thomasnet.com/profile/00095684/hyd...,tel://800-338-5339,hydraulic cylinders
7,30392369,"DLH Fluid Power, Inc.","Manufacturer*, Distributor",$5 - 9.9 Mil,1956,200-499,"Arcadia, CA",,,Manufacturer & distributor of standard & custo...,https://www.thomasnet.com/profile/30392369/dlh...,tel://626-574-7888,hydraulic cylinders
8,1274794,"Engineered Sales, Inc.",Distributor*,$250 Mil. and over,1965,10-49,Locations,,"ARO, Air-Dro, Airoyal, Anker-Holth, Aurora, Co...",Manufacturer & distributor of standard & custo...,https://www.thomasnet.com/branchloc.html?cid=1...,tel://314-878-4500,hydraulic cylinders
9,30422303,Aviation Equipment Service,Service Company*,$1 - 4.9 Mil,2011,10-49,"Houston, TX",,,"Repair services of cylinders, tri-pod jacks, a...",https://www.thomasnet.com/profile/30422303/avi...,tel://281-509-5184,hydraulic cylinders


# Meta-Harvester Class

In [ ]:
import os, re, time
import traceback

import requests
from bs4 import BeautifulSoup

import math
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore')

from multiprocessing import Pool


class ThomasnetMetaDataScraper:
    def __init__(self, **kwargs):
        self.config = kwargs.get('config')
        self.BASE_URL = "https://www.thomasnet.com/nsearch.html"
        self.base_payload = {
            "cov": "NA",
            "heading": self.config['heading'],
            "searchsource": "suppliers",
            "searchterm": self.config['keyword'],
            "what": self.config['keyword'],
            "pg": 1
        }
        self.collected_data = []
        self.metadata = None


    def find_num_pages(self,payload):
        page = requests.get(self.BASE_URL, params=payload)
        soup = BeautifulSoup(page.text,"lxml")

        total_suppliers = soup.find('p',class_="supplier-search-results__subheader").findAll('b')[-1].text
        print(f"{total_suppliers} suppliers found")

        n_suppliers = len(soup.findAll('div',class_="supplier-search-results__card"))
        print(f"{n_suppliers} found on this page")

        number_of_pages = math.ceil(float(total_suppliers)/float(n_suppliers))
        print(f"Total Pages: {number_of_pages}")
        return number_of_pages


    def generate_payload(self, num_pages:int, keyword:str):
        payloads = []
        for i in range(num_pages):
            payload = {
                "cov": "NA",
                "heading": self.config['heading'],
                "searchsource": "suppliers",
                "searchterm": keyword,
                "what": keyword,
                "pg": i+1
            }
            payloads.append(payload)
        return payloads


    @staticmethod
    def extract_data(payload):
        BASE_URL = "https://www.thomasnet.com/nsearch.html"
        def get_html(param):
            passed = False
            retry = 0
            while not passed:
                try:
                    page = BeautifulSoup(
                        requests.get(BASE_URL,params=param).text,
                        'lxml'
                    )
                    passed = True
                    return page
                except Exception as e:
                    retry = retry + 1
                    print(f'Retrying {retry}/5 times...')
                    if retry==5:
                        break
                    else:
                        time.sleep(5)
                        pass
        try:
            soup = get_html(payload)
            print(payload)
            suppliers_list = []
            suppliers = soup.findAll('div',class_="supplier-search-results__card")
            for sup in suppliers:
                card_data = {
                    "company_id":"",
                    "company_name":"",
                    "company_type":"",
                    "annual_revenue":"",
                    "year_founded":"",
                    "num_employees":"",
                    "location":"",
                    "company_url":"",
                    "brands":"",
                    "description":"",
                    "url":"",
                    "telephone":"",
                    "searchterm": payload['searchterm']
                }
                result = {"page_data": [card_data], "success": False}
                try:
                    header = sup.find('header',class_='profile-card__header')
                    card_data['company_id'] = eval(sup.get('data-impression-tracking'))['company_id']
                    card_data['company_name'] = header.find('h2',class_='profile-card__title').text.strip()
                    card_data['url'] = 'https://www.thomasnet.com'+header.find('h2',class_='profile-card__title').find('a').get('href')
                    card_data['telephone'] = sup.find('a',{'data-conversion_action':'Call'}).get('href')

                    sup_data = sup.find('div',class_='profile-card__supplier-data')
                    card_data['location'] = sup_data.find('span',class_='profile-card__location').text.replace('ico-map','').strip()
                    card_data['company_type'] = sup_data.find('span',{'data-content':'Company Type'}).text.strip()
                    try:
                        card_data['annual_revenue'] = sup_data.find('span',{'data-content':'Annual Revenue'}).text.strip()
                    except:
                        pass
                    try:
                        card_data['num_employees'] = sup_data.find('span',{'data-content':'Number of Employees'}).text.strip()
                    except:
                        pass
                    try:
                        card_data['year_founded'] = sup_data.find('span',{'data-content':'Year Founded'}).text.strip()
                    except:
                        pass

                    content = sup.find('div',class_='profile-card__content')
                    card_data['description'] = content.findAll('p')[0].text.strip()
                    card_data['company_url'] = content.find('p',{'class':'profile-card_web-link-wrap'}).find('a').get('href')
                    try:
                        card_data['brands'] = content.find('p',{'class':'profile-card__brands__body'}).text.strip()
                    except:
                        pass
                    suppliers_list.append(card_data)
                except Exception as e:
                    print("Error encountered while extraction of data\n", traceback.print_exc())
                    pass

            result = {"page_data": suppliers_list, "success": True}
            print(f"Successfully scraped page {payload['pg']}")
        except Exception as e:
            print(f"Error encountered scraping page {payload['pg']}:\n{e}")
        finally:
            return result


    def save_data(self):
        if not os.path.exists(self.config['saving_path']):
            os.makedirs(os.path.dirname(self.config['saving_path']))
        try:
            self.metadata = pd.DataFrame(self.collected_data)
            self.metadata.to_csv(self.config['saving_path'],index=False)
        except Exception as e:
            print(f"Error encountered saving metadata:\n\t{e}")


    def run(self):
        num_pages = self.find_num_pages(self.base_payload)
        list_of_payloads = self.generate_payload(num_pages,self.config['keyword'])
        try:
            pool = Pool(processes=10)
            final_result = pool.map(self.extract_data, list_of_payloads[3])
            for result in final_result:
                if result["success"]:
                    self.collected_data.extend(result["page_data"])
        except Exception as e:
            print(f"Error occurred. Closing scraping process.\n{str(e)}", traceback.print_exc())
        finally:
            # self.save_data()
            print(self.metadata)


if __name__ =='__main__':
    config = {
        "keyword": "hydraulic cylinders",
        "heading": 21650809,
        "saving_path": "data/hydraulic_cylinders/hydraulic_cylinders_suppliers_metadata.csv"
    }
    scraper = ThomasnetMetaDataScraper(
        config=config
    )
    scraper.run()